In [94]:
from bs4 import BeautifulSoup 

from webdriver_manager.chrome import ChromeDriverManager  # 자동으로 크롬드라이버 (가상드라이버) 파일 다운로드 
from selenium.webdriver.chrome.service import Service  # 다운로드된 크롬드라이버 파일 연결하기 위해 사용 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import requests

import numpy as np
import pandas as pd
import time
import re 

from IPython.display import Image 

#Word cloud
# from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image # 만약 "No module named 'PIL'" 에러가 발생하면 [ pip install Pillow==5.4.1 ] 로 라이브러리를 설치해줍니다.
from nltk.corpus import stopwords 

import warnings
warnings.filterwarnings("ignore") # 불필요한 Warning 메시지를 꺼줍니다.

하나만 테스트로 뽑아보기

In [95]:
# songid = 7896508
# url = "https://www.melon.com/song/detail.htm?songId="+ str(songid)

# headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

# web = requests.get(url,headers=headers).content
# web_page = BeautifulSoup(web, 'html.parser')

In [96]:
# # 제목
# title = web_page.find('div',{'class':'song_name'}).get_text().strip() 
# title = title.split('\t')[-1] 
# # 가수명
# artist = web_page.find('a',{'class':'artist_name'})['title'] 
# # 발매일, 장르
# date_genre = web_page.find('dl',{'class':'list'}).get_text() 
# date = date_genre.split('\n')[4] 
# genre = date_genre.split('\n')[6] 
# #가사
# lyric = web_page.find('div',{'class':'lyric'}).get_text().strip() 
# # get_text() 제거하면 br태그 같이 나옴
# lyric_2 = web_page.find('div',{'class':'lyric'}) #.get_text()#.strip() 
# # get_text() 제거하면 br태그 같이 나옴

# print(title, artist, date, genre)
# print()
# print(lyric) 
# print(type(lyric))
# print()
# print(lyric_2) 
# print(type(lyric_2))


1-1) SongID 크롤링


In [97]:
# 노래 고유 번호(song id) 가져오기 
def get_songid(gnrcode):
    service = Service(executable_path=ChromeDriverManager().install()) 
    driver = webdriver.Chrome(service = service) 
    
    # 노래 고유번호
    song_id = []
    
    # url의 startindex 변경 -> Page 변경 (10페이지, 장르당 500곡)
    for start_idx in range(1, 452,50): # 1,452,50
        songlist_url = 'https://www.melon.com/genre/song_list.htm?gnrCode=' + gnrcode \
        + '&dtlGnrCode=#params%5BgnrCode%5D=' + gnrcode \
        + '&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj' \
        + '&startIndex=' + str(start_idx)

        print(songlist_url)

        driver.get(songlist_url)
        time.sleep(3)
        web = driver.page_source
        source = BeautifulSoup(web,'html.parser')

        # 노래 고유번호인 Songid를 추출
        for song_info in source.find_all('div',{'class':'ellipsis rank01'}):
            # print('크롤링 중입니다 : ', song_info.find('a')['title'][:-2])
            id = song_info.find('a')['href']
            id = re.findall("\d+", id)[-1]  
            song_id.append(id)
        print('****다음 페이지로 넘어갑니다****')

    # 하나의 장르 크롤링 완료
    print("---------------------------------------------------")
    print("장르 랩 크롤링 완료되었습니다")

    driver.close()
    driver.quit()
    
    return song_id

SongID로 곡 정보 가져오기

In [98]:
def songid_to_info(songlist):
    #변수 담을 리스트들 생성
    titles = []
    songID = []
    artists = []
    dates = []
    genres = []
    lyrics = []

    #곡 정보 받아오기
    idx=0
    for songid in songlist:
        url = "https://www.melon.com/song/detail.htm?songId="+ songid

        # songID
        songID.append(songid)


        # print(url)

        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web = requests.get(url,headers=headers).content
        web_page = BeautifulSoup(web, 'html.parser')
        
        # print(web_page)

        #노래 제목 줄바꿈 제거
        title = web_page.find('div',{'class':'song_name'}).get_text().strip().split('\t')[-1]
        if title:
            #title = title.split('(')[0].strip() # (feat.) 등 제거
            titles.append(title)
        else:
            titles.append(' ')
        
        #가수명
        artist = web_page.find('a',{'class':'artist_name'})['title']
        if artist:
            artists.append(artist)
        else:
            artists.append(' ')
            
        #곡 발매일, 장르
        date_genre = web_page.find('dl',{'class':'list'}).get_text()
        if date_genre:
            date = date_genre.split('\n')[4]
            genre = date_genre.split('\n')[6]
            dates.append(date)
            genres.append(genre)
        else:
            dates.append(' ')
            genres.append(' ')

        #가사
        try: 
            lyric = web_page.find('div',{'class':'lyric'}) #.get_text().strip()
            if lyrics:
                lyrics.append(lyric)
            else:
                lyrics.append(' ')
        except: # 성인인증이 필요한 경우 스크래핑 불가 -> 가사 생략
            print('----{} : 성인인증이 필요한 가사입니다----'.format(title))
            lyrics.append(' ')
             

        idx += 1 #다음곡으로 넘어가기
        if idx%100 ==0:
            print(idx, '번째곡 크롤링 완료 :', title)

    info_dict = {'Title':titles, 
                 'SongID': songID,
                 'Artist':artists, 
                 'Date':dates, 
                 'Genre':genres, 
                 'Lyric':lyrics,
                }
    print(len(titles), len(artists), len(dates), len(genres), len(lyrics))
    df_song_info = pd.DataFrame(info_dict)
    return df_song_info

# genre_name 장르명 만 변경하면 됩니다!!

In [99]:
# # 개인 크롬 path 삽입 -> 로컬 chrome driver 이용했을 경우
# chrome_path = '/opt/homebrew/bin/chromedriver'

# 맡은 장르 적어주세요 
genre_name = '인디음악'

In [100]:
# 장르와 장르 id 매칭 
gnr_info = {'발라드':'GN0100', '댄스':'GN0200', '랩_힙합':'GN0300', '알앤비_소울':'GN0400',
           '인디음악':'GN0500', '록_메탈':'GN0600', '트로트':'GN0700','포크_블루스':'GN0800'}

# song_id 가져오기 
song_id = get_songid(gnr_info[genre_name])

# 노래 정보 가져오기 
df_genre_songid_to_info = songid_to_info(song_id) 

# 엑셀 반환 
filename = "{0}_500songs.xlsx".format(genre_name) # 250songs
df_genre_songid_to_info.to_excel(filename, index=False, encoding='utf-8')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/seunguk/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


https://www.melon.com/genre/song_list.htm?gnrCode=GN0500&dtlGnrCode=#params%5BgnrCode%5D=GN0500&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0500&dtlGnrCode=#params%5BgnrCode%5D=GN0500&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=51
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0500&dtlGnrCode=#params%5BgnrCode%5D=GN0500&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=101
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0500&dtlGnrCode=#params%5BgnrCode%5D=GN0500&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=151
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0500&dtlGnrCode=#params%5BgnrCode%5D=GN0500&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP

#### 엑셀 반환 
유의점: '랩/힙합' 같이 

In [101]:
df_genre_songid_to_info

,Title,SongID,Artist,Date,Genre,Lyric
0,너를 생각해,33978183,주시크 (Joosiq),2021.09.22,"R&B;/Soul, 인디음악",
1,Seoul,34930369,볼빨간사춘기,2022.04.20,"인디음악, 포크/블루스","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
2,아무래도 난,34817660,주시크 (Joosiq),2022.03.30,"R&B;/Soul, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
3,고백,33868791,멜로망스,2021.08.29,"발라드, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
4,찰나가 영원이 될 때 (The Eternal Moment),33976677,마크툽 (MAKTUB),2021.09.19,"R&B;/Soul, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
...,...,...,...,...,...,...
495,? (Prod. Color),32572358,LEON (딥상어),2020.05.06,"랩/힙합, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
496,수영해 (Virus Edit),32818005,유라 (youra),2020.07.28,"인디음악, 일렉트로니카","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
497,Wish,8298726,잔나비,2016.08.04,"인디음악, 록/메탈","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
498,Flight +82,35099560,Jimmy Brown,2022.05.15,"R&B;/Soul, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."


In [102]:
df_genre_songid_to_info.Lyric.iloc[1]

<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->
							Seoul 처음 만난 낯선 도시에서<br/>사람들 사이에 빛이 나는<br/>기타를 멘 멋진 소년과 yeah<br/>사랑에 빠진 Love Story<br/>들어볼래 지나치게도<br/>소설 같은 이야기일 거야<br/>Maybe Someday<br/>꽃잎이 떨어지는 그날에<br/>우리가 만난 그곳에서<br/>멀리서 기타 소리가 들리면 나<br/>너를 위해 노랠 부를 거야<br/>City of light<br/>빛이 가득한 이 도시에서<br/>눈을 반짝이며 꿈을 꾸는<br/>마이크를 쥔 멋진 소녀와<br/>사랑에 빠진 Love Story<br/>Maybe Someday<br/>꽃잎이 떨어지는 그날에<br/>우리가 만난 그곳에서<br/>멀리서 기타 소리가 들리면 나<br/>너를 위해 노랠 부를 거야<br/>언젠가 사랑했던 그 도시에서<br/>내 목소릴 기억한다면<br/>그때 우리 만나자 my Seoul<br/>Someday<br/>너와 나 꿈꿔왔던 그날에<br/>서로가 마주한 곳에서<br/>환하게 미소 지을 수 있다면 나<br/>너를 위해 노래해<br/>Someday 그때 그날에<br/>우리가 만난 그곳에서<br/>멀리서 네 목소리가 들리면 나<br/>너를 위해 노랠 부를 거야<br/>
</div>

---